In [1]:
from typing import Tuple, List, Union, Any, Optional, Dict, Literal, Callable
import time
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

import numpy as np
import aeon
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from aeon.datasets.tsc_datasets import univariate_equal_length, multivariate_equal_length
univariate_equal_length = sorted(list(univariate_equal_length))
multivariate_equal_length = sorted(list(multivariate_equal_length))
from aeon.datasets import load_classification
from sklearn.linear_model import RidgeCV, RidgeClassifierCV
from sklearn.metrics import accuracy_score
from tqdm import tqdm

from utils.utils import print_name, print_shape
from preprocessing.stream_transforms import normalize_mean_std_traindata, normalize_streams, augment_time, add_basepoint_zero
from random_sig_fourier import SigTensorisedRandProj
from signature import SigTransform, LogSigTransform
from features.base import TimeseriesFeatureExtractor, TabularTimeseriesFeatures, RandomGuesser
from randomized_sig import RandomizedSignature
from rocket_wrappers import RocketWrapper, MiniRocketWrapper, MultiRocketWrapper
from multirocket import MultiRocketOwn
from frozen_gpt2 import ProjTimeseriesGPT2Last, ProjTimeseriesGPT2Multipooling, ProjTimeseriesGPT2MultipoolingAndLast

np.set_printoptions(precision=3, threshold=5) # Print options

/home/nikita/Code/zephyrox/.conda/lib/python3.10/site-packages/aeon/base/__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(


In [2]:
#############################################
#######          Dataset Code         #######
#############################################

def get_aeon_dataset(
        dataset_name:str, 
        extract_path = "/home/nikita/hdd/Data/MTSC/",
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        ):
    """Loads a dataset from the UCR/UEA archive using 
    the aeon library.

    Args:
        dataset_name (str): Name of the dataset

    Returns:
        Tuple: 4-tuple of the form (X_train, y_train, X_test, y_test)
    """
    X_train, y_train = load_classification(dataset_name, split="train", extract_path=extract_path)
    X_test, y_test = load_classification(dataset_name, split="test", extract_path=extract_path)
    X_train = torch.from_numpy(X_train.transpose(0,2,1)).to(device).float().detach()
    X_test = torch.from_numpy(X_test.transpose(0,2,1)).to(device).float().detach()
    return X_train, y_train, X_test, y_test

In [3]:
# def test_get_data(idx: int = 17):
#     name = univariate_equal_length[idx]
#     X_train, y_train, X_test, y_test = get_aeon_dataset(name, device="cpu")
#     print("Dataset:", name)
#     print("idx:", idx)
#     print("X_train", X_train.shape)
#     print("X_test", X_test.shape)

# for i in range(20):
#     test_get_data(i)
#     print("\n")

In [4]:
##################################
####  Linear Model (Ridge)  ######
##################################

def train_and_test_linear(
        train_X, train_y, test_X, test_y,
        feat_extractor: TimeseriesFeatureExtractor,
        apply_augmentation:bool=True,
        normalize_features:bool=True,
        clf=RidgeClassifierCV(alphas=np.logspace(-1, 3, 20))
    ):
    # augment data
    print(train_X.shape)
    if apply_augmentation:
        train_X, test_X = normalize_mean_std_traindata(train_X, test_X)
        train_X = add_basepoint_zero(train_X)
        #train_X = augment_time(train_X)
        test_X = add_basepoint_zero(test_X)
        #test_X = augment_time(test_X)

    # fit transformer
    t0 = time.time()
    with torch.no_grad():
        feat_extractor.fit(train_X)
        feat_train_X = feat_extractor.transform(train_X).cpu().numpy()
        feat_test_X = feat_extractor.transform(test_X).cpu().numpy()
        print("feat_train_X", feat_train_X.shape)
        if normalize_features:
            feat_train_X, feat_test_X = normalize_mean_std_traindata(feat_train_X, feat_test_X)


    # feed into linear classifier
    t1 = time.time()
    clf.fit(feat_train_X, train_y)
    t2 = time.time()

    # predict
    pred = clf.predict(feat_test_X)
    test_acc = accuracy_score(test_y, pred)
    train_acc = accuracy_score(train_y, clf.predict(feat_train_X))
    alpha = clf.alpha_ if hasattr(clf, 'alpha_') else None
    return train_acc, test_acc, alpha, t1-t0, t2-t1

In [5]:
def run_allModels_singleDataset(X_train, y_train, X_test, y_test):
    max_batch = 8
    trunc_level = 4
    n_features = 1344
    n_rocket_features = 20000

    models = [
        ["Random Guesser", RandomGuesser()],
        ["Tabular", TabularTimeseriesFeatures()],
        # # ["Sig", SigTransform(trunc_level, max_batch)],
        # # ["Log Sig", LogSigTransform(trunc_level, max_batch)],
        # ["Randomized Signature", RandomizedSignature(
        #     n_features,
        #     activation = "tanh",
        #     max_batch=10,
        #     )],
        # ["TRP", SigTensorisedRandProj(
        #     trunc_level,
        #     n_features,
        #     only_last=True,
        #     method="linear",
        #     max_batch=max_batch,
        #     )],
        # ["TRP rbf", SigTensorisedRandProj(
        #     trunc_level,
        #     n_features,
        #     only_last=True,
        #     method="RBF",
        #     sigma_rbf=1.0,
        #     max_batch=max_batch,
        #     )],
        # ["concat TRP", SigTensorisedRandProj(
        #     trunc_level,
        #     n_features // (trunc_level-1),
        #     only_last=False,
        #     method="linear",
        #     max_batch=max_batch,
        #     )],
        # ["concat TRP rbf", SigTensorisedRandProj(
        #     trunc_level,
        #     n_features // (trunc_level-1),
        #     only_last=False,
        #     method="RBF",
        #     sigma_rbf=1.0,
        #     max_batch=max_batch,
            # )],
        # ["Rocket", RocketWrapper(
        #     n_rocket_features
        #     )],
        ["MiniRocket", MiniRocketWrapper(
            n_rocket_features
            )],
        ["MultiRocket", MultiRocketWrapper(
            n_rocket_features
            )],
        # ["MyOwnMultiRocket", MultiRocketOwn(
        #     n_rocket_features,
        #     max_batch,
        #     )],
        ["GPT2 Last", ProjTimeseriesGPT2Last(
            n_rocket_features,
            max_batch,
            )],
        ["GPT2 Pooling", ProjTimeseriesGPT2Multipooling(
            n_rocket_features,
            max_batch,
            )],
        ["GPT2 Last+Pooling", ProjTimeseriesGPT2MultipoolingAndLast(
            n_rocket_features,
            max_batch,
            )],
        ]

    # Run experiments
    model_names = [name for (name, _) in models]
    results_ridge = []
    for name, model in models:
        print("name", name)
        result = train_and_test_linear(
            X_train, y_train, X_test, y_test, model
            )
        results_ridge.append(result)
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        print()
    
    return model_names, results_ridge

In [6]:
def run_dataset(dataset_name:str):
    X_train, y_train, X_test, y_test = get_aeon_dataset(dataset_name)
    X_train, X_test = normalize_streams(X_train, X_test, max_T=1000)
    model_names, results_ridge = run_allModels_singleDataset(X_train, y_train, X_test, y_test)
    return model_names, results_ridge

#model_names, results_ridge = run_dataset(univariate_equal_length[9])

In [7]:
# from rocket_wrappers import RocketWrapper, MiniRocketWrapper, MultiRocketWrapper

# for n_features in [100, 1344, 4000, 6000, 8000, 10000]:
#     rocket = RocketWrapper(n_features)
#     mini_rocket = MiniRocketWrapper(n_features)
#     multi_rocket = MultiRocketWrapper(n_features)
#     own = MultiRocketOwn(n_features)
#     gpt = ProjTimeseriesGPT2MultipoolingAndLast(n_features)
    
#     print("n_features:", n_features)
    
#     # Generate random input
#     input_shape = (2, 25, 1)  # N T D
#     X = torch.randn(input_shape).float().to("cuda")
    
#     # Fit to random input
#     print(rocket.fit_transform(X).shape)
#     print(mini_rocket.fit_transform(X).shape)
#     print(multi_rocket.fit_transform(X).shape)
#     print(own.fit_transform(X).shape)
#     print(gpt.fit_transform(X).shape)
#     print()

In [8]:
def run_allModels_allData(datasets: List[str]):
    #run experiments
    experiments = {}
    failed = {}
    for dataset_name in tqdm(datasets):
        t0 = time.time()
        # try:
        print(dataset_name)
        X_train, y_train, X_test, y_test = get_aeon_dataset(dataset_name)
        X_train, X_test = normalize_streams(X_train, X_test, max_T=1000)
        N_train = X_train.shape[0]
        N_test = X_test.shape[0]
        T = X_train.shape[1]
        D = X_train.shape[2]
        if N_train<=2000:
            results = run_allModels_singleDataset(
                X_train, y_train, X_test, y_test
                )
            experiments[dataset_name] = results
        # except Exception as e:
        #     print(f"Error: {e}")
        #     failed[dataset_name] = e
        print("Elapsed time", time.time()-t0)
    
    #parse results
    # Define the attributes and methods
    attributes = ["ACC_train", "ACC_test", "alpha", "time_transform", "time_fit"]
    
    # Extract model_names from d_res
    model_names = next(iter(experiments.values()))[0]

    # Create and save DataFrames for each attribute and method
    for attribute in attributes:
        df = pd.DataFrame(columns=model_names)
        for dataset_name, (model_names, results_ridge) in experiments.items():
            values = [res[attributes.index(attribute)] for res in results_ridge]
            df.loc[dataset_name] = values

        # Save the DataFrame
        print(df)
        df.to_pickle(f"MTSC_{attribute}_results.pkl")

    return experiments, failed

In [9]:
used_by_paper = [
    "EthanolConcentration",
    #"FaceDetection",
    "Handwriting",
    "Heartbeat",
    #"JapaneseVowels", #unequal length
    "PEMS-SF",
    "SelfRegulationSCP1",
    "SelfRegulationSCP2",
    #"SpokenArabicDigits", #unequal length
    "UWaveGestureLibrary",
]


# run_allModels_allData(used_by_paper)
run_allModels_allData(used_by_paper)
# run_allModels_allData(univariate_equal_length[0:10])

  0%|          | 0/7 [00:00<?, ?it/s]

EthanolConcentration
name Random Guesser
torch.Size([261, 875, 3])
feat_train_X (261, 2)

name Tabular
torch.Size([261, 875, 3])
feat_train_X (261, 2628)

name MiniRocket
torch.Size([261, 875, 3])
feat_train_X (261, 19992)

name MultiRocket
torch.Size([261, 875, 3])
feat_train_X (261, 19488)

name GPT2 Last
torch.Size([261, 875, 3])
split 0
split 1
split 2
split 3
split 4
split 5
split 6
split 7
split 8
split 9
split 10
split 11
split 12
split 13
split 14
split 15
split 16
split 17
split 18
split 19
split 20
split 21
split 22
split 23
split 24
split 25
split 0
split 1
split 2
split 3
split 4
split 5
split 6
split 7
split 8
split 9
split 10
split 11
split 12
split 13
split 14
split 15
split 16
split 17
split 18
split 19
split 20
split 21
split 22
split 23
split 24
split 25
split 0
split 1
split 2
split 3
split 4
split 5
split 6
split 7
split 8
split 9
split 10
split 11
split 12
split 13
split 14
split 15
split 16
split 17
split 18
split 19
split 20
split 21
split 22
split 23
split 24
sp

 14%|█▍        | 1/7 [44:34<4:27:29, 2674.90s/it]


Elapsed time 2674.897433757782
Handwriting
name Random Guesser
torch.Size([150, 152, 3])
feat_train_X (150, 2)

name Tabular
torch.Size([150, 152, 3])
feat_train_X (150, 459)

name MiniRocket
torch.Size([150, 152, 3])
feat_train_X (150, 19992)

name MultiRocket
torch.Size([150, 152, 3])
feat_train_X (150, 19488)

name GPT2 Last
torch.Size([150, 152, 3])
split 0
split 1
split 2
split 3
split 4
split 5
split 6
split 7
split 8
split 9
split 10
split 11
split 12
split 13
split 14
split 15
split 16
split 17
split 18
split 19
split 20
split 21
split 22
split 23
split 24
split 25
split 0
split 1
split 2
split 3
split 4
split 5
split 6
split 7
split 8
split 9
split 10
split 11
split 12
split 13
split 14
split 15
split 16
split 17
split 18
split 19
split 20
split 21
split 22
split 23
split 24
split 25
split 0
split 1
split 2
split 3
split 4
split 5
split 6
split 7
split 8
split 9
split 10
split 11
split 12
split 13
split 14
split 15
split 16
split 17
split 18
split 19
split 20
split 21
split 2

 29%|██▊       | 2/7 [57:11<2:08:51, 1546.22s/it]

...

 86%|████████▌ | 6/7 [2:32:54<26:21, 1581.04s/it]


Elapsed time 1264.011759519577
UWaveGestureLibrary
name Random Guesser
torch.Size([120, 315, 3])
feat_train_X (120, 2)

name Tabular
torch.Size([120, 315, 3])
feat_train_X (120, 948)

name MiniRocket
torch.Size([120, 315, 3])
feat_train_X (120, 19992)

name MultiRocket
torch.Size([120, 315, 3])
feat_train_X (120, 19488)

name GPT2 Last
torch.Size([120, 315, 3])
split 0
split 1
split 2
split 3
split 4
split 5
split 6
split 7
split 8
split 9
split 10
split 11
split 12
split 13
split 14
split 15
split 16
split 17
split 18
split 19
split 20
split 21
split 22
split 23
split 24
split 25
split 0
split 1
split 2
split 3
split 4
split 5
split 6
split 7
split 8
split 9
split 10
split 11
split 12
split 13
split 14
split 15
split 16
split 17
split 18
split 19
split 20
split 21
split 22
split 23
split 24
split 25
split 0
split 1
split 2
split 3
split 4
split 5
split 6
split 7
split 8
split 9
split 10
split 11
split 12
split 13
split 14
split 15
split 16
split 17
split 18
split 19
split 20
split 21

100%|██████████| 7/7 [2:45:49<00:00, 1421.39s/it]


Elapsed time 775.6963832378387
                      Random Guesser   Tabular  MiniRocket  MultiRocket  \
EthanolConcentration        0.295019  0.724138         1.0          1.0   
Handwriting                 0.073333  0.886667         1.0          1.0   
Heartbeat                   0.720588  0.995098         1.0          1.0   
PEMS-SF                     0.183521  1.000000         1.0          1.0   
SelfRegulationSCP1          0.537313  0.944030         1.0          1.0   
SelfRegulationSCP2          0.555000  0.860000         1.0          1.0   
UWaveGestureLibrary         0.175000  0.933333         1.0          1.0   

                      GPT2 Last  GPT2 Pooling  GPT2 Last+Pooling  
EthanolConcentration   0.747126      0.992337           0.992337  
Handwriting            1.000000      1.000000           1.000000  
Heartbeat              1.000000      1.000000           1.000000  
PEMS-SF                1.000000      1.000000           1.000000  
SelfRegulationSCP1     0.996269 

({'EthanolConcentration': (['Random Guesser',
    'Tabular',
    'MiniRocket',
    'MultiRocket',
    'GPT2 Last',
    'GPT2 Pooling',
    'GPT2 Last+Pooling'],
   [(0.2950191570881226,
     0.28517110266159695,
     379.2690190732246,
     0.0022935867309570312,
     0.0054874420166015625),
    (0.7241379310344828,
     0.5817490494296578,
     33.59818286283781,
     0.01283407211303711,
     0.053229331970214844),
    (1.0, 0.44106463878326996, 1000.0, 4.697098731994629, 0.1516425609588623),
    (1.0, 0.4524714828897338, 1000.0, 9.26844310760498, 0.16668295860290527),
    (0.7471264367816092,
     0.33840304182509506,
     1000.0,
     1808.0766191482544,
     0.2119755744934082),
    (0.9923371647509579,
     0.28517110266159695,
     1000.0,
     468.19943046569824,
     0.17144179344177246),
    (0.9923371647509579,
     0.2889733840304182,
     1000.0,
     382.2953853607178,
     0.20133066177368164)]),
  'Handwriting': (['Random Guesser',
    'Tabular',
    'MiniRocket',
    '

In [10]:
# Define the attributes and methods
attributes = ["ACC_train", "ACC_test", "time_transform", "time_fit", "alpha"]
#data_dir = "https://github.com/nikitazozoulenko/zephyrox/raw/main/Data/TSER/"
data_dir = ""
# Load and store the DataFrames for each attribute and method
dfs = {}
for attribute in attributes:
    filename = f"MTSC_{attribute}_results.pkl"
    print(data_dir+filename)
    df = pd.read_pickle(data_dir + filename)
    dfs[attribute] = df

MTSC_ACC_train_results.pkl
MTSC_ACC_test_results.pkl
MTSC_time_transform_results.pkl
MTSC_time_fit_results.pkl
MTSC_alpha_results.pkl


In [11]:
dfs["alpha"]

,Random Guesser,Tabular,MiniRocket,MultiRocket,GPT2 Last,GPT2 Pooling,GPT2 Last+Pooling
EthanolConcentration,379.269019,33.598183,1000.0,1000.0,1000.000000,1000.000000,1000.000000
Handwriting,1000.000000,1000.000000,1000.0,1000.0,1000.000000,1000.000000,1000.000000
Heartbeat,1000.000000,1000.000000,1000.0,1000.0,1000.000000,1000.000000,1000.000000
PEMS-SF,1000.000000,0.100000,0.1,0.1,379.269019,88.586679,143.844989
SelfRegulationSCP1,1000.000000,1000.000000,1000.0,1000.0,1000.000000,1000.000000,1000.000000
SelfRegulationSCP2,143.844989,1000.000000,1000.0,1000.0,1000.000000,1000.000000,1000.000000
UWaveGestureLibrary,379.269019,1000.000000,0.1,0.1,54.555948,0.100000,0.100000


In [12]:
dfs["ACC_test"]

,Random Guesser,Tabular,MiniRocket,MultiRocket,GPT2 Last,GPT2 Pooling,GPT2 Last+Pooling
EthanolConcentration,0.285171,0.581749,0.441065,0.452471,0.338403,0.285171,0.288973
Handwriting,0.032941,0.191765,0.414118,0.377647,0.232941,0.295294,0.290588
Heartbeat,0.721951,0.721951,0.760976,0.717073,0.687805,0.741463,0.731707
PEMS-SF,0.144509,0.867052,0.855491,0.815029,0.815029,0.820809,0.826590
SelfRegulationSCP1,0.505119,0.856655,0.866894,0.938567,0.795222,0.856655,0.856655
SelfRegulationSCP2,0.488889,0.511111,0.477778,0.538889,0.433333,0.483333,0.494444
UWaveGestureLibrary,0.131250,0.821875,0.928125,0.946875,0.715625,0.881250,0.871875


In [13]:
dfs["ACC_train"]

,Random Guesser,Tabular,MiniRocket,MultiRocket,GPT2 Last,GPT2 Pooling,GPT2 Last+Pooling
EthanolConcentration,0.295019,0.724138,1.0,1.0,0.747126,0.992337,0.992337
Handwriting,0.073333,0.886667,1.0,1.0,1.000000,1.000000,1.000000
Heartbeat,0.720588,0.995098,1.0,1.0,1.000000,1.000000,1.000000
PEMS-SF,0.183521,1.000000,1.0,1.0,1.000000,1.000000,1.000000
SelfRegulationSCP1,0.537313,0.944030,1.0,1.0,0.996269,1.000000,1.000000
SelfRegulationSCP2,0.555000,0.860000,1.0,1.0,0.990000,1.000000,1.000000
UWaveGestureLibrary,0.175000,0.933333,1.0,1.0,1.000000,1.000000,1.000000


In [14]:
dfs["time_transform"]

,Random Guesser,Tabular,MiniRocket,MultiRocket,GPT2 Last,GPT2 Pooling,GPT2 Last+Pooling
EthanolConcentration,0.002294,0.012834,4.697099,9.268443,1808.076619,468.199430,382.295385
Handwriting,0.000452,0.002759,2.477001,4.705044,527.862874,119.443940,100.291033
Heartbeat,0.004016,0.047515,4.974844,11.644222,581.399488,142.640975,118.430839
PEMS-SF,0.069635,0.390357,13.136400,24.517610,196.746147,50.318577,41.700425
SelfRegulationSCP1,0.000580,0.011820,6.937174,12.467006,2218.143689,571.403452,460.805086
SelfRegulationSCP2,0.000409,0.013652,3.364138,8.445850,897.721761,193.698731,158.835717
UWaveGestureLibrary,0.000517,0.003627,2.031578,3.804518,544.580890,122.293059,101.746807
